In [34]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from copy import copy
from openpyxl.cell.cell import MergedCell
from datetime import datetime


Load you files in the next code block. And edit the date input for the date you would like to update.

In [35]:
# === CONFIGURATION ===

target_date_input = '2024-03-16'
report_file = "Daily production report March 2025.xlsx"
# === Load Source Data ===
source_file = "03 March  25 Gross Gen.xlsx"
gross_wb = load_workbook("Gross Gen. Summary 2025.xlsx", data_only=True)

In [36]:


# Usage in your script:
target_date = pd.to_datetime(target_date_input)
date_str = target_date.strftime("%dth %B")  # Formats as "9th March"


summary_df = pd.read_excel(source_file, sheet_name="Summary", header=None)

# Make a full copy for lookups
full_df = summary_df.copy()

# Extract data section and clean it
data_df = summary_df.iloc[9:].copy()

In [37]:
print("\n=== Checking if 'Total MWH' is in raw summary_df ===")
matches = summary_df.apply(lambda row: row.astype(str).str.contains("Total MWH", case=False, na=False).any(), axis=1)
total_mwh_rows = summary_df[matches]
print(total_mwh_rows)


# Reset index
data_df = data_df.reset_index(drop=True)

# Rename columns manually
data_df.columns = [
    "DATE", "DG SET .1", "DG SET .2", "DG SET .3", "DG SET .4", 
    "DG SET .5", "STG", "DAILY TOTAL MWH", "PLANT GROSS", "ENG GROSS", "STG2"
]

# Improved date parsing - handle multiple formats
data_df["DATE_PARSED"] = pd.to_datetime(
    data_df["DATE"], 
    errors='coerce',
    dayfirst=True  # Important for European date formats
).dt.normalize()  # Remove time component

# Debug: Show available dates
print("Available dates in source data:")
print(data_df["DATE_PARSED"].dropna().dt.strftime("%Y-%m-%d").unique())


=== Checking if 'Total MWH' is in raw summary_df ===
           0                                      1         2         3   \
7        DATE  Daily Gross Generation per unit (MWH)       NaN       NaN   
40  Total MWH                               1700.832  1427.616  1824.576   

          4         5      6                7          8          9         10  
7        NaN       NaN    NaN  DAILY TOTAL MWH        NaN        NaN       NaN  
40  1514.992  1304.256  229.6         8001.872  8001872.0  7772272.0  229600.0  
Available dates in source data:
['2024-03-01' '2024-03-02' '2024-03-03' '2024-03-04' '2024-03-05'
 '2024-03-06' '2024-03-07' '2024-03-08' '2024-03-09' '2024-03-10'
 '2024-03-11' '2024-03-12' '2024-03-13' '2024-03-14' '2024-03-15'
 '2024-03-16' '2024-03-17' '2024-03-18' '2024-03-19' '2024-03-20'
 '2024-03-21' '2024-03-22' '2024-03-23' '2024-03-24' '2024-03-25'
 '2024-03-26' '2024-03-27' '2024-03-28' '2024-03-29' '2024-03-30'
 '2024-03-31']


In [38]:
# Find the MWH value for the date
daily_row = data_df[data_df["DATE_PARSED"] == target_date]
mwh_value = daily_row["DAILY TOTAL MWH"].values[0] if not daily_row.empty else None

def extract_monthly_mwh(summary_df, total_label="Total MWH"):
    print("\n=== Debugging extract_monthly_mwh() ===")
    
    # Search for row that contains the label (excluding header rows)
    mask = summary_df.apply(lambda row: row.astype(str).str.strip().str.lower().str.contains(total_label.lower()).any(), axis=1)
    # Skip first 10 rows which likely contain headers
    mask[:10] = False
    total_row = summary_df[mask]
    
    if total_row.empty:
        print(f"No row found containing '{total_label}'")
        return None
    
    print(f"Found row with label '{total_label}':")
    print(total_row)
    
    # Find the "DAILY TOTAL MWH" column index from header row (row 7)
    daily_total_col = None
    header_row = summary_df.iloc[7]  # Headers are in row 8 (0-based index 7)
    for col_idx, val in enumerate(header_row):
        if str(val).strip().upper() == "DAILY TOTAL MWH":
            daily_total_col = col_idx
            print(f"Found 'DAILY TOTAL MWH' at column index {col_idx}")
            break
    
    if daily_total_col is not None:
        # Get the value from Total MWH row (not header row)
        val = total_row.iloc[0, daily_total_col]
        print(f"Value in same column of Total MWH row: {val} (type: {type(val)})")
        try:
            float_val = float(val)
            print(f"Monthly MWH from '{total_label}' row: {float_val}")
            return float_val
        except (ValueError, TypeError):
            print(f"Could not convert value '{val}' to float")
    
    # Fallback: Check all columns for numeric values in the actual Total MWH row
    print("\nChecking all columns for numeric values:")
    for col in reversed(total_row.columns):
        val = total_row.iloc[0][col]
        print(f"Column {col}: {val} (type: {type(val)})")
        try:
            if pd.notna(val):
                float_val = float(val)
                print(f"Found numeric value: {float_val}")
                return float_val
        except (ValueError, TypeError):
            continue
    
    print(f"No numeric value found in row labeled '{total_label}'")
    return None

In [39]:
# === Write to Daily Report ===
if mwh_value is not None:
    wb = load_workbook(report_file)
    
    # Check if a sheet with the given date_str exists
    if date_str in wb.sheetnames:
        # If the sheet exists, update the value in cell B8
        ws = wb[date_str]
        ws["B8"] = mwh_value
        wb.save("Daily production report March 2025.xlsx")
        print(f"Inserted {mwh_value} into sheet '{date_str}' cell B8.")
    
    # If the sheet doesn't exist, create a new sheet with the date_str name
    else:
        # Find an existing sheet to duplicate (use the first sheet here as an example)
        sheet_to_duplicate = wb.worksheets[2]  # You can adjust this if you want a specific sheet
        
        # Create a new sheet by copying the content of the original sheet
        new_sheet = wb.copy_worksheet(sheet_to_duplicate)

        
        
        # Rename the new sheet to the current date_str
        new_sheet.title = date_str

        for row in sheet_to_duplicate.iter_rows():
            for cell in row:
            # Skip MergedCells that are not the actual top-left anchor
                if isinstance(cell, MergedCell):
                    continue

                new_cell = new_sheet.cell(row=cell.row, column=cell.column)
                                        
                # Copy styles
                if cell.has_style:
                    new_cell.font = copy(cell.font)
                    new_cell.border = copy(cell.border)
                    new_cell.fill = copy(cell.fill)
                    new_cell.number_format = copy(cell.number_format)
                    new_cell.protection = copy(cell.protection)
                    new_cell.alignment = copy(cell.alignment)

            
                # Copy only formulas or static labels (no user-filled values)
                if cell.data_type == 'f':
                    new_cell.value = f"={cell.value}"
                elif isinstance(cell.value, str) and cell.value.strip() != "":
                    new_cell.value = cell.value  # Copy headers/static text
                else:
                    new_cell.value = None  # Clear user-entered numbers or blanks

        # Copy merged cell ranges
        for merged_range in sheet_to_duplicate.merged_cells.ranges:
            new_sheet.merge_cells(str(merged_range))
        # Copy column widths
        for col in sheet_to_duplicate.column_dimensions:
            new_sheet.column_dimensions[col].width = sheet_to_duplicate.column_dimensions[col].width

        # Copy row heights
        for row_dim in sheet_to_duplicate.row_dimensions:
            new_sheet.row_dimensions[row_dim].height = sheet_to_duplicate.row_dimensions[row_dim].height

        #Rename cell B3 with correct name
        new_sheet["B3"].value = f"{date_str} 2025"


        
        # Assign the new sheet to `ws` so we can insert value below
        ws = new_sheet
        print(f"New sheet '{date_str}' has been created.")

    # Insert the value into B8, regardless of whether the sheet existed or was just created
    ws["B8"] = mwh_value

    # Insert the monthly total MWH into B9
    monthly_mwh = extract_monthly_mwh(summary_df)  # Call your earlier function here
    if monthly_mwh is not None:
        ws["B9"] = monthly_mwh
        print(f"Inserted monthly MWH value {monthly_mwh} into cell B9.")
    else:
        print("Monthly MWH value not found. Skipping B9 insertion.")
    
    # Step: Extract Annual MWH from Gross Gen Summary 2025
    try:
       
        gross_ws = gross_wb.active  # Or specify by name if needed: gross_wb["SomeSheetName"]
        annual_mwh_value = float(gross_ws["H17"].value)
        print(f"Annual MWH value extracted from H17: {annual_mwh_value}")
    except Exception as e:
        annual_mwh_value = None
        print(f"Error extracting annual MWH value: {e}")

    # Step: Add to B10 in the Daily Report
    if annual_mwh_value is not None:
        ws["B10"] = annual_mwh_value
        print(f"Inserted annual MWH value {annual_mwh_value} into cell B10.")
    else:
        print("Annual MWH value not found. Skipping B10 insertion.")


    wb.save("Daily production report March 2025 - updated.xlsx")

    print(f"Inserted {mwh_value} into sheet '{date_str}' cell B8.")
else:
    print("No data found for that date.")

    



New sheet '16th March' has been created.

=== Debugging extract_monthly_mwh() ===
Found row with label 'Total MWH':
           0         1         2         3         4         5      6   \
40  Total MWH  1700.832  1427.616  1824.576  1514.992  1304.256  229.6   

          7          8          9         10  
40  8001.872  8001872.0  7772272.0  229600.0  
Found 'DAILY TOTAL MWH' at column index 7
Value in same column of Total MWH row: 8001.872000000076 (type: <class 'float'>)
Monthly MWH from 'Total MWH' row: 8001.872000000076
Inserted monthly MWH value 8001.872000000076 into cell B9.
Annual MWH value extracted from H17: 32133.028000000064
Inserted annual MWH value 32133.028000000064 into cell B10.
Inserted 288.23200000007637 into sheet '16th March' cell B8.
